In [ ]:
import akshare as ak
from coursea import *
await init()
from omicron.extension import find_runs
import pandas as pd

import arrow


: 

In [12]:
now = datetime.datetime.now().date()
start = tf.day_shift(now, -1000)

start_date = start.strftime("%Y%m%d")
end_date = now.strftime("%Y%m%d")

print(start_date)
print(end_date)
# 通过akshare获取中证1000日线数据（近1000天）
bars = ak.index_zh_a_hist(symbol="000852", start_date=start_date, end_date=end_date)

bars.rename(columns = {
    "日期": "frame",
    "开盘": "open",
    "最高": "high",
    "最低": "low",
    "收盘": "close"
}, inplace=True)

bars["frame"] = bars.frame.apply(lambda x: arrow.get(x).date())

close = bars.close
returns = close.pct_change()
v, s, l = find_runs(returns <= 0)

cum_neg_returns = []
for vi, si, li in zip(v, s, l):
    if vi and li > 1:
        cum_neg_returns.append((bars.frame[si-1], bars.frame[si + li - 1], close[si + li - 1]/close[si-1] - 1))
        
r = pd.DataFrame(cum_neg_returns, columns=["start", "end", "cnr"])
r

20200303
20240416


,start,end,cnr
0,2020-03-05,2020-03-09,-0.042150
1,2020-03-10,2020-03-18,-0.104779
2,2020-03-25,2020-03-30,-0.040948
3,2020-04-09,2020-04-13,-0.036343
4,2020-04-22,2020-04-29,-0.047872
...,...,...,...
112,2024-01-11,2024-01-22,-0.108763
113,2024-01-25,2024-02-05,-0.192455
114,2024-03-20,2024-03-27,-0.069431
115,2024-04-01,2024-04-08,-0.028629


In [13]:
# 3/27这一次下跌0.0694，此后继续下跌的概率

p_decline = r.cnr.le(-0.06943).mean()
p_decline

0.08547008547008547

In [14]:
# 4/16这一次下跌0.058，此后继续下跌的概率

p_decline = r.cnr.le(-0.058228).mean()
p_decline

0.1452991452991453

In [19]:
import numpy as np
for loss in np.linspace(-0.05, -0.1, 10):
    print(f"下跌幅度{loss:.1%}, 反弹概率: {1- r.cnr.le(loss).mean():.1%}")

下跌幅度-5.0%, 反弹概率: 82.9%
下跌幅度-5.6%, 反弹概率: 83.8%
下跌幅度-6.1%, 反弹概率: 88.0%
下跌幅度-6.7%, 反弹概率: 90.6%
下跌幅度-7.2%, 反弹概率: 92.3%
下跌幅度-7.8%, 反弹概率: 94.9%
下跌幅度-8.3%, 反弹概率: 94.9%
下跌幅度-8.9%, 反弹概率: 96.6%
下跌幅度-9.4%, 反弹概率: 96.6%
下跌幅度-10.0%, 反弹概率: 96.6%
